In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim 
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import gurobipy as gp
from gurobipy import GRB
from tqdm import tqdm 
from datasets import all_data 
from helper import *  
from models import LogisticRegressionModel

In [2]:
merit_attrs_dict = {'lsac': [2,3], 'crime': [54,61], 'compas': [395,399]}

In [3]:
dataset_name = 'lsac'
sens_attribute = "Race"
nominal = False
merit = True

In [4]:
train_x = pd.read_csv('train_X_'+dataset_name+'.csv')
train_x.columns

Index(['decile1b', 'decile3', 'lsat', 'ugpa', 'zfygpa', 'zgpa', 'fulltime',
       'fam_inc', 'male', 'tier', 'race'],
      dtype='object')

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# set parameters 
num_epochs = 20 
lr_theta = 1e-3 
lr_z = 1e-1 
epsilon = 0.01 
delta = 0.2 

In [7]:
dataset = all_data(sens_attribute=sens_attribute, train=True, dataset_name=dataset_name)
merit_attrs = merit_attrs_dict[dataset_name] 
train_loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)
train_X_1, train_sens_1, train_y_1 = dataset.get_values()
flag = dataset.get_flag()
train_x_bar = dataset.get_xbar()
dataset_features = dataset.get_columns()
merit_attrs_strs = [dataset_features[i] for i in merit_attrs]

In [8]:
X, s, y, idx = next(iter(train_loader))
num_feat = X.shape[1] 
model = LogisticRegressionModel(num_feat,1) 
model = model.to(device) 
model_parameters = filter(lambda p: p.requires_grad, model.parameters()) 
opt = optim.Adam(model.parameters(), lr=lr_theta)

In [9]:
z_list = []
z0_train = torch.tensor([0 for i in range(len(train_y_1))])
z0_train = z0_train.to(device)
train_X_1 = torch.Tensor(train_X_1)
train_sens_1 = torch.Tensor(list(train_sens_1))
train_y_1 = torch.Tensor(list(train_y_1))
z0_train = proj_z_structured(z0_train, train_sens_1, train_y_1, train_X_1, train_x_bar, flag, epsilon, delta, merit_attrs)

Academic license - for non-commercial use only - expires 2023-10-25
Using license file /Users/thodoris/gurobi.lic


In [11]:
for _ in range(num_epochs):
      if nominal:
          train_err, train_loss = epoch_nominal(train_loader, model, opt=opt)
      else:
          train_err, train_loss, z = epoch_flipped_structured(train_loader, model, z0_train, train_sens_1, train_y_1, train_X_1, train_x_bar, flag, merit_attrs, epsilon=epsilon, delta=delta, lr_z=lr_z, opt=opt)
          z0_train = z

      print(*("{:.6f}".format(i) for i in (train_err, train_loss)), sep="\t")

state = {
          'epoch': num_epochs,
          'state_dict': model.state_dict(),
          'optimizer': opt.state_dict(),
       }

if nominal: 
    nom_str = 'nominal'
else: 
    nom_str = 'flipped'

if sens_attribute == 'Gender':
    sens_str = 'gender'
else:
    sens_str = 'race'

savepath = dataset_name+'_'+sens_str+'_checkpoint_'+nom_str+'.pth'
torch.save(state,savepath)

100%|██████████| 227/227 [00:00<00:00, 570.40it/s]

Academic license - for non-commercial use only - expires 2023-10-25


Using license file /Users/thodoris/gurobi.lic


 69%|██████▊   | 156/227 [00:00<00:00, 766.24it/s]

0.112911	0.326617


...


 67%|██████▋   | 151/227 [00:00<00:00, 729.72it/s]

0.108381	0.303356


100%|██████████| 227/227 [00:00<00:00, 730.36it/s]

Academic license - for non-commercial use only - expires 2023-10-25
Using license file /Users/thodoris/gurobi.lic


0.107489	0.304616
